# VacationPy

### Import dependencies and key

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key and access gmaps
from api_keys import g_key
gmaps.configure(api_key=g_key)

# Data from Part 1
data_file = "../output/cities.csv"

### Import Part I results into DataFrame

In [2]:
df = pd.read_csv(data_file)
df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,bubaque,36,GW,1578091256,64,11.28,-15.83,78.94,11.74
1,arcelia,92,MX,1578091258,41,18.32,-100.28,79.56,1.54
2,wenchi,15,GH,1578091259,13,7.74,-2.10,71.26,5.23
3,samarai,59,PG,1578091260,63,-10.62,150.67,86.29,3.13
4,iranshahr,59,IR,1578091261,75,27.21,60.69,60.12,1.01


In [3]:
df.dtypes

City           object
Cloudiness      int64
Country        object
Date            int64
Humidity        int64
Lat           float64
Lng           float64
Max Temp      float64
Wind Speed    float64
dtype: object

### Humidity Heatmap

In [4]:
locations = df[['Lat','Lng']]
humidities = df['Humidity']

# Create and format figure
fig = gmaps.figure(
    center=(20,0),
    zoom_level=2,
    layout={
        'width': '1000px',
        'height': '500px'
    }
)

# create heat layer and add to plot
heat_map = gmaps.heatmap_layer(
    locations,
    weights=humidities,
    dissipating=False,
    max_intensity=max(humidities),
    point_radius=2
)
fig.add_layer(heat_map)

# show map
fig

Figure(layout=FigureLayout(height='500px', width='1000px'))

### Create new DataFrame fitting weather criteria

In [5]:
# Cities with ideal weather
cities_ideal = df.loc[
    (df['Max Temp']>70) & (df['Max Temp']<80) &
    (df['Wind Speed']<10) & 
    (df['Cloudiness']==0),
    ['City','Country','Lat','Lng']
]

print(f'Query returns {len(cities_ideal)} cities with ideal weather')
cities_ideal

Query returns 4 cities with ideal weather


,City,Country,Lat,Lng
8,chiredzi,ZW,-21.05,31.67
101,barrow,AR,-38.31,-60.23
204,naze,NG,5.43,7.07
247,dawei,MM,14.07,98.19


### Hotel Map

In [6]:
# make a copy of dataframe to `hotel_df`
# add 'Hotel Name' column
hotel_df = cities_ideal.copy()
hotel_df['Hotel Name'] = pd.Series()
hotel_df

,City,Country,Lat,Lng,Hotel Name
8,chiredzi,ZW,-21.05,31.67,NaN
101,barrow,AR,-38.31,-60.23,NaN
204,naze,NG,5.43,7.07,NaN
247,dawei,MM,14.07,98.19,NaN


In [7]:
# Google Places API parameter
params = {
    'type': 'lodging',
    'radius': 8000,      #meters
    'key': g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Loop through each ideal city
for index, row in hotel_df.iterrows():
    # add lat/long to parameters
    params['location'] = f"{row['Lat']},{row['Lng']}"
    
    # API request
    print(f"Looking for hotel in {row['City']}...")
    results = requests.get(base_url, params=params).json()['results']

    # Store first result into dataframe
    try:
        hotel_df.loc[index,'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print(f"No hotels found within {params['radius']} meters")

hotel_df    
    

Looking for hotel in chiredzi...
Looking for hotel in barrow...
Looking for hotel in naze...
Looking for hotel in dawei...


,City,Country,Lat,Lng,Hotel Name
8,chiredzi,ZW,-21.05,31.67,The Nesbitt Arms
101,barrow,AR,-38.31,-60.23,Segura
204,naze,NG,5.43,7.07,Oxygen Holiday Resort
247,dawei,MM,14.07,98.19,Hotel Dawei


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(
    locations,
    info_box_content=hotel_info
)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='500px', width='1000px'))